## **SemEval 2020 Task 4 - Commonsense Validation and Explanation**

# **Subtask A**

**Accuracy on eval_set: 88.46%**

**Accuracy on test_set: 87.7%**

 

---



---



**Installing Pre-requiste Libraries**

In [1]:
!pip install simpletransformers

     |████████████████████████████████| 204kB 14.9MB/s 
     |████████████████████████████████| 1.4MB 31.1MB/s 
     |████████████████████████████████| 2.9MB 27.3MB/s 
     |████████████████████████████████| 317kB 54.8MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 1.1MB 49.0MB/s 
     |████████████████████████████████| 1.8MB 48.4MB/s 
     |████████████████████████████████| 7.4MB 13.3MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
     |████████████████████████████████| 890kB 46.2MB/s 
     |████████████████████████████████| 163kB 55.1MB/s 
     |████████████████████████████████| 102kB 14.5MB/s 
     |████████████████████████████████| 133kB 59.0MB/s 
     |████████████████████████████████| 102kB 13.4MB/s 
     |████████████████████████████████| 112kB 50.7MB/s 
     |████████████████████████████████| 4.5MB 45.9MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 122kB 54.6MB/

**Connecting the backup drive**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Inserting Prerequiste Libraries**

In [2]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn
import numpy as np
import torch

**Fetching Dataset**

In [3]:
from google.colab import files
uploaded = files.upload()

Saving subtaskA_answers_all.csv to subtaskA_answers_all.csv
Saving subtaskA_data_all.csv to subtaskA_data_all.csv
Saving subtaskA_dev_data.csv to subtaskA_dev_data.csv
Saving subtaskA_gold_answers 2.csv to subtaskA_gold_answers 2.csv
Saving subtaskA_gold_answers.csv to subtaskA_gold_answers.csv
Saving subtaskA_test_data.csv to subtaskA_test_data.csv


In [4]:
subtaskA_train_data = pd.read_csv('subtaskA_data_all.csv', header=None)
#subtaskA_train_data = subtaskA_train_data[1:]    #because first row contains sent0,sent1,id etc
subtaskA_answers_for_train = pd.read_csv('subtaskA_answers_all.csv', header=None)
subtaskA_dev_data = pd.read_csv('subtaskA_dev_data.csv', header=None)
#subtaskA_dev_data = subtaskA_dev_data[1:]        #because first row contains sent0,sent1,id etc
subtaskA_answers_for_dev = pd.read_csv('subtaskA_gold_answers.csv', header=None)
subtaskA_test_data = pd.read_csv('subtaskA_test_data.csv', header=None)
#subtaskA_test_data = subtaskA_test_data[1:]      #because first row contains sent0,sent1,id etc
subtaskA_answers_for_test = pd.read_csv('subtaskA_gold_answers 2.csv', header=None)

**Preprocessing on subtaskA_dataset**

In [5]:
train_s1 = subtaskA_train_data[:][1]
train_s2 = subtaskA_train_data[:][2]


In [6]:
subtaskA_answers_for_train

,0,1
0,0,0
1,1,0
2,2,1
3,3,1
4,4,1
...,...,...
9995,9995,0
9996,9996,0
9997,9997,1
9998,9998,1


In [7]:
train_labels=subtaskA_answers_for_train[:][1]
train_labels=np.array(train_labels)
train_labels

array([0, 0, 1, ..., 1, 1, 0])

In [8]:
train_s1=train_s1[1:]
train_s1=np.array(train_s1)
train_s1

array(['He poured orange juice on his cereal.', 'He drinks apple.',
       'Jeff ran a mile today', ...,
       'Harry went to the barbershop to have his hair cut',
       'Reilly is sleeping on the couch', 'I have a desk on my lamp'],
      dtype=object)

In [9]:
train_s2=train_s2[1:]
train_s2=np.array(train_s2)
train_s2

array(['He poured milk on his cereal.', 'He drinks milk.',
       'Jeff ran 100,000 miles today', ...,
       'Harry went to the barbershop to have his glasses repaired',
       'Reilly is sleeping on the window', 'I have a lamp on my desk'],
      dtype=object)

In [10]:
print(len(train_s1))
print(len(train_s2))
print(len(train_labels))

10000
10000
10000


In [11]:
train_s1 = pd.Series(train_s1)
train_s2 = pd.Series(train_s2)
train_labels = pd.Series(train_labels)

In [12]:
train_s1 = train_s1.to_frame()
train_s2 = train_s2.to_frame()
train_labels = train_labels.to_frame()

train_A = pd.concat([train_s1, train_s2, train_labels], axis=1)


In [13]:
train_A.columns

Int64Index([0, 0, 0], dtype='int64')

In [14]:
train_A

,0,0,0
0,He poured orange juice on his cereal.,He poured milk on his cereal.,0
1,He drinks apple.,He drinks milk.,0
2,Jeff ran a mile today,"Jeff ran 100,000 miles today",1
3,A mosquito stings me,I sting a mosquito,1
4,A niece is a person.,A giraffe is a person.,1
...,...,...,...
9995,Mark ate a big bitter cherry pie,Mark ate a big sweet cherry pie,0
9996,Gloria wears a cat on her head,Gloria wears a hat on her head,0
9997,Harry went to the barbershop to have his hair cut,Harry went to the barbershop to have his glass...,1
9998,Reilly is sleeping on the couch,Reilly is sleeping on the window,1


In [15]:
train_A.columns = ['text_a','text_b','labels']

In [16]:
train_A

,text_a,text_b,labels
0,He poured orange juice on his cereal.,He poured milk on his cereal.,0
1,He drinks apple.,He drinks milk.,0
2,Jeff ran a mile today,"Jeff ran 100,000 miles today",1
3,A mosquito stings me,I sting a mosquito,1
4,A niece is a person.,A giraffe is a person.,1
...,...,...,...
9995,Mark ate a big bitter cherry pie,Mark ate a big sweet cherry pie,0
9996,Gloria wears a cat on her head,Gloria wears a hat on her head,0
9997,Harry went to the barbershop to have his hair cut,Harry went to the barbershop to have his glass...,1
9998,Reilly is sleeping on the couch,Reilly is sleeping on the window,1


In [17]:
subtaskA_dev_data

,0,1,2
0,id,sent0,sent1
1,1363,"Summer in North America is great for skiing, ...","Summer in North America is great for swimming,..."
2,392,You can use detergent to dye your hair.,You can use bleach to dye your hair.
3,540,passing your driving license exams requires st...,passing your university exams requires studyin...
4,3,The hangers bought the closet,The closet got hangers
...,...,...,...
993,700,THE TEMPERATURE IS EMOTIONAL,THE TEMPERATURE IS WARM
994,355,Blended up fruit makes a nacho,Blended up fruit makes a smoothie
995,1550,Dogs falls from the sky,Rain falls from the sky
996,1364,He put a dinosaur in the fridge,He put a chicken in the fridge


In [18]:
subtaskA_answers_for_dev

,0,1
0,1363,0
1,392,0
2,540,0
3,3,0
4,560,1
...,...,...
992,700,0
993,355,0
994,1550,0
995,1364,0


In [19]:
subtaskA_test_data

,0,1,2
0,id,sent0,sent1
1,1175,He loves to stroll at the park with his bed,He loves to stroll at the park with his dog.
2,452,The inverter was able to power the continent.,The inverter was able to power the house
3,275,The chef put extra lemons on the pizza.,The chef put extra mushrooms on the pizza.
4,869,sugar is used to make coffee sour,sugar is used to make coffee sweet
...,...,...,...
996,1114,"If it had rained, you would got wet.","If it is a sunny day, you would got wet."
997,8,ice hockey is a sport,ice hockey is a financial institution
998,1945,He put water without a container in the freeze...,He put a watermelon in the freezer for 24 hours
999,1053,The desert has sand that you can drink.,"The desert is very dry, so bring water when yo..."


In [20]:
subtaskA_answers_for_test

,0,1
0,1175,0
1,452,0
2,275,0
3,869,0
4,50,1
...,...,...
995,1114,1
996,8,1
997,1945,0
998,1053,0


In [21]:
dev_s1 = subtaskA_dev_data[:][1]
dev_s2 = subtaskA_dev_data[:][2]

In [22]:
dev_s1

0                                                  sent0
1      Summer in North America is great for skiing,  ...
2                You can use detergent to dye your hair.
3      passing your driving license exams requires st...
4                          The hangers bought the closet
                             ...                        
993                         THE TEMPERATURE IS EMOTIONAL
994                       Blended up fruit makes a nacho
995                              Dogs falls from the sky
996                      He put a dinosaur in the fridge
997    The cleaner is in charge of the money at the s...
Name: 1, Length: 998, dtype: object

In [23]:
dev_labels=subtaskA_answers_for_dev[:][1]
dev_labels=np.array(dev_labels)


In [24]:
dev_s1=dev_s1[1:]
dev_s1=np.array(dev_s1)
dev_s1

array(['Summer in North America is great for skiing,  snowshoeing,  and making a snowman.',
       'You can use detergent to dye your hair.',
       'passing your driving license exams requires studying for your classes.',
       'The hangers bought the closet', 'coffee takes sleep',
       'The cat used the litter box.', 'He wrote an exam in pen',
       'she gave birth to a baby', 'Chicken are found on a farm.',
       'A stick wants to have sex',
       'I went to the restaurant and we saw a monkey.',
       'A circle always has a radius.',
       'Mike moves the desk with his fin.',
       'The stove was cleaned with a cleaner.', 'the bear ate a fish',
       'the titanic flew into the sky after it hit an iceberg',
       'A bear was driving a car in Canada',
       'I use the money to buy tickets for the bus',
       'the sandwich contains meat', 'he takes his car into his bedroom',
       'The repairer can be repaired by a car.',
       'He spends leaves for dinner.', 'Dragonfly 

In [25]:
dev_s2=dev_s2[1:]
dev_s2=np.array(dev_s2)
dev_s2

array(['Summer in North America is great for swimming,  boating, and fishing.',
       'You can use bleach to dye your hair.',
       'passing your university exams requires studying for your classes.',
       'The closet got hangers', 'coffee depresses people',
       'The cat used the toilet.', 'He wrote an exam in knife',
       'he gave birth to a baby', 'Chicken can swim in water.',
       'A person wants to have sex',
       'I went to the zoo and we saw a monkey.',
       'A circle has four corners.', 'Mike moves the desk with his arms.',
       'The stove was cleaned with a mop.', 'the bear ate a car',
       'the titanic sank after it hit an iceberg',
       'He saw a bear during his trip to Canada',
       'i use my biscuits to buy tickets for bus',
       'the sandwich contains sadness',
       'he takes his toy into his bedroom',
       'The car can be repaired by a repairer.',
       'He spends $100 for dinner.', 'shoes can fly',
       'the bed was warm and soft', 'Boats 

In [26]:
print(len(dev_s1))
print(len(dev_s2))
print(len(dev_labels))

997
997
997


In [27]:
dev_s1 = pd.Series(dev_s1)
dev_s2 = pd.Series(dev_s2)
dev_labels = pd.Series(dev_labels)

In [28]:
dev_s1 = dev_s1.to_frame()
dev_s2 = dev_s2.to_frame()
dev_labels = dev_labels.to_frame()

dev_A = pd.concat([dev_s1, dev_s2, dev_labels], axis=1)



In [29]:
dev_A.columns = ['text_a','text_b','labels']

dev_A

,text_a,text_b,labels
0,"Summer in North America is great for skiing, ...","Summer in North America is great for swimming,...",0
1,You can use detergent to dye your hair.,You can use bleach to dye your hair.,0
2,passing your driving license exams requires st...,passing your university exams requires studyin...,0
3,The hangers bought the closet,The closet got hangers,0
4,coffee takes sleep,coffee depresses people,1
...,...,...,...
992,THE TEMPERATURE IS EMOTIONAL,THE TEMPERATURE IS WARM,0
993,Blended up fruit makes a nacho,Blended up fruit makes a smoothie,0
994,Dogs falls from the sky,Rain falls from the sky,0
995,He put a dinosaur in the fridge,He put a chicken in the fridge,0


In [30]:
test_s1 = subtaskA_test_data[:][1]
test_s2 = subtaskA_test_data[:][2]

In [31]:
test_labels=subtaskA_answers_for_test[:][1]
test_labels=np.array(test_labels)

In [32]:
test_s1=test_s1[1:]
test_s1=np.array(test_s1)
test_s2=test_s2[1:]
test_s2=np.array(test_s2)

In [33]:
print(len(test_s1))
print(len(test_s2))
print(len(test_labels))

1000
1000
1000


In [34]:
test_s1 = pd.Series(test_s1)
test_s2 = pd.Series(test_s2)
test_labels = pd.Series(test_labels)

In [35]:
test_s1 = test_s1.to_frame()
test_s2 = test_s2.to_frame()
test_labels = test_labels.to_frame()

test_A = pd.concat([test_s1, test_s2, test_labels], axis=1)

test_A.columns = ['text_a','text_b','labels']

test_A

,text_a,text_b,labels
0,He loves to stroll at the park with his bed,He loves to stroll at the park with his dog.,0
1,The inverter was able to power the continent.,The inverter was able to power the house,0
2,The chef put extra lemons on the pizza.,The chef put extra mushrooms on the pizza.,0
3,sugar is used to make coffee sour,sugar is used to make coffee sweet,0
4,There are beautiful flowers here and there in ...,There are beautiful planes here and there in t...,1
...,...,...,...
995,"If it had rained, you would got wet.","If it is a sunny day, you would got wet.",1
996,ice hockey is a sport,ice hockey is a financial institution,1
997,He put water without a container in the freeze...,He put a watermelon in the freezer for 24 hours,0
998,The desert has sand that you can drink.,"The desert is very dry, so bring water when yo...",0


**Installing Simpletransformers**

In [36]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [48]:
train_args={
    'output_dir': "/content/gdrive/MyDrive/outputs" ,
    'best_model_dir': "/content/gdrive/MyDrive/outputs/best_model" ,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 20,
    'no_save': True
}

In [49]:
torch.cuda.current_device()

0

In [50]:
torch.cuda.device(0)

In [51]:
torch.cuda.device_count()

1

In [52]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [53]:
torch.cuda.is_available()

True

In [54]:
# Create a ClassificationModel
model = ClassificationModel('roberta', 'roberta-base', num_labels=2, use_cuda=True, cuda_device=0, args=train_args)
print(train_A.head())

INFO:filelock:Lock 139842882219536 acquired on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock


INFO:filelock:Lock 139842882219536 released on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
INFO:filelock:Lock 139841424614904 acquired on /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7.lock


INFO:filelock:Lock 139841424614904 released on /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7.lock


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

INFO:filelock:Lock 139841300773800 released on /root/.cache/huggingface/transformers/d3ccdbfeb9aaa747ef20432d4976c32ee3fa69663b379deb253ccfce2bb1fdc5.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab.lock
INFO:filelock:Lock 139841318302944 acquired on /root/.cache/huggingface/transformers/cafdecc90fcab17011e12ac813dd574b4b3fea39da6dd817813efa010262ff3f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


INFO:filelock:Lock 139841318302944 released on /root/.cache/huggingface/transformers/cafdecc90fcab17011e12ac813dd574b4b3fea39da6dd817813efa010262ff3f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock



                                  text_a                         text_b  labels
0  He poured orange juice on his cereal.  He poured milk on his cereal.       0
1                       He drinks apple.                He drinks milk.       0
2                  Jeff ran a mile today   Jeff ran 100,000 miles today       1
3                   A mosquito stings me             I sting a mosquito       1
4                   A niece is a person.         A giraffe is a person.       1


**Training SimpleTransformer Classfication Model on subtaskA_training_set**

In [55]:
# Train the model
model.train_model(train_A, eval_df=dev_A)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to /content/gdrive/MyDrive/outputs.


(25000, 0.12830499666656328)

**Results on Dev Set**

In [56]:
result, model_outputs, wrong_predictions = model.eval_model(dev_A, acc=sklearn.metrics.accuracy_score)
print(result)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.7690318169771119, 'tp': 414, 'tn': 468, 'fp': 50, 'fn': 65, 'acc': 0.8846539618856569, 'eval_loss': 1.0306488707725294}



{'mcc': 0.7690318169771119, 'tp': 414, 'tn': 468, 'fp': 50, 'fn': 65, 'acc': 0.8846539618856569, 'eval_loss': 1.0306488707725294}


In [57]:
predictions, raw_outputs = model.predict([["He loves to stroll at the park with his bed", "He loves to stroll at the park with his dog."]])
print(predictions)
print(raw_outputs)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.



[0]
[[ 5.4375     -5.49609375]]


In [60]:
test_s1 = list(test_A['text_a'])
test_s2 = list(test_A['text_b'])
test_labels = list(test_A['labels'])

**Results on test set**

In [61]:
result, model_outputs, wrong_predictions = model.eval_model(test_A, acc=sklearn.metrics.accuracy_score)
print(result)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.7544384461927134, 'tp': 420, 'tn': 457, 'fp': 51, 'fn': 72, 'acc': 0.877, 'eval_loss': 1.0707464737138652}



{'mcc': 0.7544384461927134, 'tp': 420, 'tn': 457, 'fp': 51, 'fn': 72, 'acc': 0.877, 'eval_loss': 1.0707464737138652}


In [62]:
print("Alhamdulilah")

Alhamdulilah


In [65]:
result['acc']

0.877

In [66]:
print(model_outputs)

[[ 5.4375     -5.49609375]
 [ 5.4453125  -5.5       ]
 [ 4.015625   -4.15234375]
 ...
 [ 5.4375     -5.49609375]
 [ 5.4140625  -5.484375  ]
 [ 5.4609375  -5.50390625]]


In [67]:
print(wrong_predictions)

[<simpletransformers.classification.classification_utils.InputExample object at 0x7f2e2a986f60>, <simpletransformers.classification.classification_utils.InputExample object at 0x7f2e2a585a58>, <simpletransformers.classification.classification_utils.InputExample object at 0x7f2e2a4932b0>, <simpletransformers.classification.classification_utils.InputExample object at 0x7f2e2a4b4e10>, <simpletransformers.classification.classification_utils.InputExample object at 0x7f2e2a4d0fd0>, <simpletransformers.classification.classification_utils.InputExample object at 0x7f2e2a350ef0>, <simpletransformers.classification.classification_utils.InputExample object at 0x7f2e2a37d278>, <simpletransformers.classification.classification_utils.InputExample object at 0x7f2e2a37df98>, <simpletransformers.classification.classification_utils.InputExample object at 0x7f2e2a11fd30>, <simpletransformers.classification.classification_utils.InputExample object at 0x7f2e2a102fd0>, <simpletransformers.classification.clas

In [68]:
model_outputs.shape

(1000, 2)

In [69]:
type(model_outputs)

numpy.ndarray

In [79]:
est = []
a=0
b=0
for i in range(1000):
  a = model_outputs[i][0]
  b = model_outputs[i][1]
  if (a>b):
    est.append(0)
  else :
    est.append(1)

In [80]:
cor = 0
for i in range(len(test_labels)):
  if (est[i]==test_labels[i]):
    cor += 1

print("test_set accuracy:", cor/len(test_labels))

test_set accuracy: 0.877


**Saving Results of test_set**

In [81]:
import csv
with open('/content/gdrive/My Drive/Anas Azam Bhatti/subtaskA_answers.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for i in range(1000):
      writer.writerow([i,est[i]])
